# Introduction

Le **traduction automatique**, tout comme d'autres tâches de NLP,  a connu des évolutions majeures ces dernières années grâce au _Deep Learning_.

Ce notebook présente un exemple simple de traduction, utilisant des **réseaux de neurones récurrents (RNN)**.

Le but est d'entraîner un modèle à **traduire différents formats de dates vers un format unique**.

# Imports

In [ ]:
import pandas as pd
import numpy as np

from string import ascii_lowercase, digits
from typing import List, Dict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.utils import plot_model

# Données

Pour cet exercice, on a généré au préalable un dataset de 25 000 dates différentes entre le `01/01/1800` et le `31/12/2050`.

On a réparti ces données en deux datasets:
- `train_data`: 10 000 pour l'entraînement du modèle
- `test_data`: 15 000 pour l'évaluation

Chaque dataset a deux colonnes:
- `input`: la date dans l'un des différents formats sources
- `target`: la date au format cible

Commençons par charger les données et un aperçu de celles-ci:

In [ ]:
train_data = pd.read_csv("../../datasets/dates/train_dates.csv", sep=";")
test_data = pd.read_csv("../../datasets/dates/test_dates.csv", sep=";")

train_data.head()

On vérifie ensuite la taille des datasets, et on s'assure qu'il n'y a aucune date en commun entre `train_data` et `test_data`:

In [ ]:
print("Taille du dataset d'entraînement:", len(train_data))
print("Taille du dataset d'évaluation:", len(test_data))
print("Nombre de dates communes entre `train_data` et `test_data`:", 
      len(set(train_data['target'].values).intersection(set(test_data['target'].values))))

# Feature engineering

Les données qu'on manipule sont des **données textuelles**. 

Afin de pouvoir les traiter avec un modèle de Machine Learning, il faut d'abord réussir à les **transformer en _features_ (caractéristiques) numériques**. C'est l'objectif de cette section.

## Tokenisation

Un texte d'entrée est d'abord **_tokenisé_**, c'est-à-dire découpé en une **séquence d'unités de texte (_tokens_)**. 

Pour cet exemple simple, nos unités de texte seront **les caractères**. La traduction consistera donc à prédire une séquence de caractères à partir d'une autre séquence de caractères.

Il est à noter que dans les vrais modèles de traduction, les _tokens_ sont plutôt des mots, ou des parties de mots (préfixes, racines, suffixes...).

In [ ]:
def tokenize(date_str: str) -> str:
    return [c for c in date_str]

In [ ]:
example_date = "Thursday, 28 Aug. 1958"
tokenized_date = tokenize(example_date)

print("Exemple de date:")
print(example_date)
print()
print("Résultat de la tokenisation:")
print(tokenized_date)

## Vocabulaire

L'ensemble des _tokens_ distincts dans la langue d'entrée constitue le **vocabulaire**.

On a choisi de faire une **_tokenisation_ niveau caractères** afin d'avoir un vocabulaire de taille réduite. Pour le réduire encore plus, on va remplacer tous les caractères spéciaux autres que le tiret `-` par des espaces ` `, et toutes les majuscules par des minuscules. Un vocabulaire réduit simplifie la tâche de traduction, et peut donc potentiellement accélérer la convergence du modèle.

- Le vocabulaire d'entrée se limite alors aux caractères alphanumériques minuscules, en plus de l'espace ` ` et du tiret `-`.
- Le vocabulaire cible est encore plus réduit, il est constitué des chiffres de `0` à `9` et du tiret `-`.

In [ ]:
input_vocabulary = list(digits) + list(ascii_lowercase) + [' ', '-']
target_vocabulary = list(digits) + ['-']

print("Vocabulaire d'entrée:")
print(input_vocabulary)
print()
print("Vocabulaire cible:")
print(target_vocabulary)

## Encodage

Un encodage possible du texte en _features_ numériques est de remplacer chaque caractère par son index dans le vocabulaire. 

Pour simplifier l'encodage, on va construire un mapping `(caractère > index)` pour le vocabulaire d'entrée, et de même pour le vocabulaire cible.

Pour simplifier le décodage des prédictions du modèle (qui vont être numériques), on va également construire un mapping `(index > caractère)` pour le vocabulaire cible.

In [ ]:
input_vocab_char_to_idx = {char: idx for (idx, char) in enumerate(input_vocabulary)}
target_vocab_char_to_idx = {char: idx for (idx, char) in enumerate(target_vocabulary)}
target_vocab_idx_to_char = {idx: char for (idx, char) in enumerate(target_vocabulary)}

print("Mapping (caractère > index) pour le vocabulaire d'entrée:")
print(input_vocab_char_to_idx)
print()
print("Mapping (caractère > index) pour le vocabulaire cible:")
print(target_vocab_char_to_idx)
print()
print("Mapping (index > caractère) pour le vocabulaire cible:")
print(target_vocab_idx_to_char)

Implémentons maintenant les fonctions d'encodage et de décodage pour une séquence entière. La fonction d'encodage inclut la tokenisation comme première étape.

In [ ]:
def encode(date_str: str, char_to_idx_mapping: Dict[str, int]) -> List[int]:
    tokenized_date = tokenize(date_str)
    encoding = []
    for c in tokenized_date:
        c = c.lower()
        c = c if c in char_to_idx_mapping else ' '
        encoding.append(char_to_idx_mapping[c])
    return encoding

def decode(encoded_text: List[int], idx_to_char_mapping: Dict[int, str]) -> str:
    return ''.join([idx_to_char_mapping[idx] for idx in encoded_text])

In [ ]:
encoded_date = encode(example_date, input_vocab_char_to_idx)
example_predicted_date = [2, 6, 10, 1, 1, 10, 2, 0, 2, 1]
decoded_date = decode(example_predicted_date, target_vocab_idx_to_char)

print("Exemple de date d'entrée:")
print(example_date)
print()
print("Résultat de l'encodage:")
print(encoded_date)
print("----------------------------------------------------------------------------------")
print("Exemple de date prédite au format encodé:")
print(example_predicted_date)
print()
print("Résultat du décodage:")
print(decoded_date)

L'encodage sous forme d'une séquence d'indexes donne une représentation numérique du texte, mais présente quelques inconvénients. 

Il instaure une "relation d'ordre" entre les différents tokens. Par exemple, le token `t` est associé à l'index `29`, alors que le token `d` est associé à l'index `13`.

Cela peut être problématique dans certains problèmes, et rendre l'apprentissage du modèle plus difficile. On optera donc pour une représentation différente, le **one-hot encoding**, qui permet d'éliminer cette notion d'ordre entre les différentes classes.

In [ ]:
def one_hot(date_str: str, char_to_idx_mapping: Dict[str, int]) -> np.ndarray:
    encoded_date = encode(date_str, char_to_idx_mapping)
    one_hot_encoding = np.zeros((len(encoded_date), len(char_to_idx_mapping)))
    for i, idx in enumerate(encoded_date):
        one_hot_encoding[i, idx] = 1
    return one_hot_encoding

In [ ]:
sample_date = "26-11-2021"
seq_encoded_date = encode(sample_date, target_vocab_char_to_idx)
one_hot_encoded_date = one_hot(sample_date, target_vocab_char_to_idx)

print("Exemple de date:")
print(sample_date)
print()
print(f"Encodage au format séquence d'indexes:")
print(seq_encoded_date)
print("Taille encodage format séquence:", len(seq_encoded_date))
print()
print(f"Encodage one-hot:")
print(one_hot_encoded_date)
print("Taille encodage format one-hot:", one_hot_encoded_date.shape)

In [ ]:
sample_date = "Thursday, 1985/01/17"
seq_encoded_date = encode(sample_date, input_vocab_char_to_idx)
one_hot_encoded_date = one_hot(sample_date, input_vocab_char_to_idx)

print("Exemple de date:")
print(sample_date)
print()
print(f"Encodage au format séquence d'indexes:")
print(seq_encoded_date)
print("Taille encodage format séquence:", len(seq_encoded_date))
print()
print(f"Encodage one-hot:")
print(one_hot_encoded_date)
print("Taille encodage format one-hot:", one_hot_encoded_date.shape)

Les modèles RNN s'entraînent sur des séquences de taille fixe et prédisent des séquences de taille fixe. Or nos dates d'entrée ont des tailles différentes. 

On doit donc faire en sorte que toutes les séquences d'entrée aient la même taille. Pour ce faire, on va utiliser du **padding**, c'est-à-dire rajouter des zéros en début de chaque chaîne de caractère, pour obtenir une longueur uniforme.

In [ ]:
def pad(one_hot_encoded: np.ndarray, max_len: int) -> np.ndarray:
    sentence_len = one_hot_encoded.shape[0]
    vocab_size = one_hot_encoded.shape[1]
    to_add = max_len - sentence_len
    return np.vstack([
        np.zeros((to_add, vocab_size)),
        one_hot_encoded
    ])

In [ ]:
print("Longueur maximale des exemples d'entraînement:", train_data['input'].str.len().max())
print("Longueur maximale des exemples d'évaluation:", test_data['input'].str.len().max())

On va donc appliquer un padding à toutes nos séquences d'entrée, pour qu'elles aient toute une longueur de 28.

Ecrivons une fonction de preprocessing globale qui enchaîne **one-hot encoding** et **zéro-padding**.

In [ ]:
def preprocess(date_str: str, char_to_idx_mapping: Dict[str, int], max_len: int):
    return pad(one_hot(date_str, char_to_idx_mapping), max_len)

In [ ]:
sample_date = "Thursday, 1985/01/17"
preprocessed_date = preprocess(sample_date, input_vocab_char_to_idx, 28)

print("Exemple de date:")
print(sample_date)
print()
print(f"Encodage one-hot et padding:")
print(preprocessed_date)
print("Taille de la représentation numérique:", preprocessed_date.shape)

Il n'y a plus qu'à appliquer cette même transformation à toutes les dates de notre dataset pour uniformiser les entrées et les sorties du modèle:

In [ ]:
X_train = np.vstack(train_data['input'].map(
    lambda d: np.expand_dims(preprocess(d, input_vocab_char_to_idx, 28), 0)
))
y_train = np.vstack(train_data['target'].map(
    lambda d: np.expand_dims(preprocess(d, target_vocab_char_to_idx, 10), 0)
))

In [ ]:
X_test = np.vstack(test_data['input'].map(
    lambda d: np.expand_dims(preprocess(d, input_vocab_char_to_idx, 28), 0)
))
y_test = np.vstack(test_data['target'].map(
    lambda d: np.expand_dims(preprocess(d, target_vocab_char_to_idx, 10), 0)
))

In [ ]:
print("TRAIN - Données d'entrée:", X_train.shape)
print("TRAIN - Données de sortie:", y_train.shape)
print("TEST - Données d'entrée:", X_test.shape)
print("TEST - Données de sortie:", y_test.shape)

# Modèle

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(28, 38)))
model.add(RepeatVector(10))
model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(11, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
plot_model(model, show_shapes=True, dpi=150, rankdir='TB')

In [ ]:
model.summary(line_length=100)

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.4)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
TEST_DATE = "Ven. 26 nov. 2021"

prediction = model.predict(
    np.expand_dims(pad(to_numeric_representation(TEST_DATE, input_vocabulary), 28), 0)
)

print(''.join([reverse_target_vocabulary[idx] for idx in prediction.argmax(axis=-1)[0]]))